In [ ]:
!pip install pycocotools

In [ ]:
from copy import deepcopy
import json
import random
import time
from pathlib import Path
import sys

import cv2
import numpy as np
import pandas as pd
import torch
import tqdm
from torch.utils import data
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.nn import functional as fnn

import matplotlib.pyplot as plt
import matplotlib.patches as patches

sys.path.insert(1, '../input/metric-tools-car-places')
from engine import evaluate

# Подготовка датасета и модели детекции номеров

In [ ]:
class CarPlatesDatasetWithRectangularBoxes(data.Dataset):
    def __init__(self, root, transforms, split='train', train_size=0.7, test_size=0.2):
        super(CarPlatesDatasetWithRectangularBoxes, self).__init__()
        self.root = Path(root)
        self.train_size = train_size
        
        self.image_names = []
        self.image_ids = []
        self.image_boxes = []
        self.image_texts = []
        self.box_areas = []
        
        self.transforms = transforms
        
        if split in ['train', 'val', 'test']:
            plates_filename = self.root / 'train.json'
            with open(plates_filename) as f:
                json_data = json.load(f)
            train_test_border = int(len(json_data) * train_size) + 1 # граница между train и test
            test_val_border = train_test_border + int(len(json_data) * test_size) + 1 # граница между test и valid 
            if split == 'train': data_range = (0, train_test_border)
            elif split == 'test': data_range = (train_test_border, test_val_border)
            else: data_range = (test_val_border, len(json_data))
            self.load_data(json_data[data_range[0]:data_range[1]]) # загружаем названия файлов и разметку
            return

        raise NotImplemented(f'Unknown split: {split}')
        
    def load_data(self, json_data):
        for i, sample in enumerate(json_data):
            if sample['file'] == 'train/25632.bmp':
                continue
            self.image_names.append(self.root / sample['file'])
            self.image_ids.append(torch.Tensor([i]))
            boxes = []
            texts = []
            areas = []
            for box in sample['nums']:
                points = np.array(box['box'])
                x_0 = np.min([points[0][0], points[3][0]])
                y_0 = np.min([points[0][1], points[1][1]])
                x_1 = np.max([points[1][0], points[2][0]])
                y_1 = np.max([points[2][1], points[3][1]])
                
                if x_0 > x_1:
                    x_1, x_0 = x_0, x_1
                if y_0 > y_1:
                    y_1, y_0 = y_0, y_1
                boxes.append([x_0, y_0, x_1, y_1])
                
                texts.append(box['text'])
                areas.append(np.abs(x_0 - x_1) * np.abs(y_0 - y_1))
            boxes = torch.FloatTensor(boxes)
            areas = torch.FloatTensor(areas)
            self.image_boxes.append(boxes)
            self.image_texts.append(texts)
            self.box_areas.append(areas)
        
    
    def load_test_data(self, plates_filename, split, train_size):
        df = pd.read_csv(plates_filename, usecols=['file_name'])
        for row in df.iterrows():
            self.image_names.append(self.root / row[1][0])
        self.image_boxes = None
        self.image_texts = None
        self.box_areas = None
         
    
    def __getitem__(self, idx):
        target = {}
        if self.image_boxes is not None:
            boxes = self.image_boxes[idx].clone()
            areas = self.box_areas[idx].clone()
            num_boxes = boxes.shape[0]
            target['boxes'] = boxes
            target['area'] = areas
            target['labels'] = torch.LongTensor([1] * num_boxes)
            target['image_id'] = self.image_ids[idx].clone()
            target['iscrowd'] = torch.Tensor([False] * num_boxes)

        image = cv2.imread(str(self.image_names[idx]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image)
        return image, target

    def __len__(self):
        return len(self.image_names)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def create_model(device):
    model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
    num_classes = 2  
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model.to(device)

def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
transformations= transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
                    ])

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

train_dataset = CarPlatesDatasetWithRectangularBoxes('/kaggle/input/car-plates-ocr/data', transformations, 'train')
val_dataset = CarPlatesDatasetWithRectangularBoxes('/kaggle/input/car-plates-ocr/data', transformations, 'val')
test_dataset = CarPlatesDatasetWithRectangularBoxes('/kaggle/input/car-plates-ocr/data', transformations, 'test')

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=collate_fn)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=2, shuffle=False, num_workers=4,
    collate_fn=collate_fn)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=2, shuffle=False, num_workers=4,
    collate_fn=collate_fn)

# Обучение модели детекции

In [ ]:
# Часть кода взята из  pytorch utils
for optimize in [torch.optim.SGD]:
    for learning_rate in [0.005, 0.001]:
        model = create_model(device)
        
        params = [p for p in model.parameters() if p.requires_grad]
        optimizer = optimize(params, lr=learning_rate, weight_decay=0.0005)

        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                       step_size=3,
                                                       gamma=0.1)

        num_epochs = 1

        for epoch in range(num_epochs):
            model.train()

            for images, targets in tqdm.tqdm(train_loader):
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())

                optimizer.zero_grad()
                losses.backward()
                optimizer.step()

            batch_losses = []
            for images, targets in tqdm.tqdm(val_loader):
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                batch_losses.append(losses.item())
                optimizer.zero_grad()

            batch_losses = np.array(batch_losses)
            batch_losses = batch_losses[np.isfinite(batch_losses)]
            print(f'Valid_loss: {np.mean(batch_losses)}')
            lr_scheduler.step()
            
            with open(f'{optimize} {learning_rate}', 'wb') as fp:
                torch.save(model.state_dict(), fp)
             
            print(f"{optimize} and {learning_rate}: {evaluate(model, test_loader, device=device)}")
            

print("That's it!")

# Проверка качества детекции


In [ ]:
unnormalize_1 = transforms.Normalize(mean=[-0.485, -0.456, -0.406],
                                         std=[1, 1, 1])
unnormalize_2 = transforms.Normalize(mean=[0, 0, 0],
                                         std=[1/0.229, 1/0.224, 1/0.225])
unnormalize = transforms.Compose([unnormalize_2, unnormalize_1])


In [ ]:
def detach_dict(pred):
    return{k:v.detach().cpu() for (k,v) in pred.items()}

In [ ]:
start = 0

images = []
for i in range(start, start + 1):
    images.append(test_dataset[i][0].to(device))

In [ ]:
model.eval()
preds = model(images)
preds = [detach_dict(pred) for pred in preds]

In [ ]:
fig,ax = plt.subplots(1, 2, figsize = (20, 8))

for i in range(1):
    image = unnormalize(images[i].clone().cpu())
    ax[i].imshow(image.numpy().transpose([1,2,0]))
    for box in preds[i]['boxes']:
        box = box.detach().cpu().numpy()
        rect = patches.Rectangle((box[0],box[1]),box[2]-box[0],box[3]-box[1],linewidth=1,edgecolor='r',facecolor='none')
        ax[i].add_patch(rect)

plt.show()

# Выделение номеров с картинок

In [ ]:
x0 = int(box[0])-20
x1 = int(box[2])+20
y0 = int(box[1])-20
y1 = int(box[3])+20

plt.imshow(images[0].cpu().numpy().transpose([1,2,0])[y0:y1, x0:x1])

In [ ]:
im = unnormalize(images[0]).cpu().numpy().transpose([1,2,0])[y0:y1, x0:x1]

In [ ]:
type(im)

In [ ]:
plt.imshow(im)

In [ ]:
plt.imsave('plate.jpg',im)

In [ ]:
import easyocr

In [ ]:
reader = easyocr.Reader(['ru','en'])
res = reader.readtext('plate.jpg', decoder = 'beamsearch', allowlist = 'УКЕНХВАРОСМИТ1234567890RUS')

In [ ]:
plate_num = res[0][1]
plate_score = res[0][2]

In [ ]:
fig, ax = plt.subplots() 
image = unnormalize(images[0].clone().cpu())
ax.imshow(image.numpy().transpose([1,2,0]))
rect = patches.Rectangle((x0,y0),x1-x0,y1-y0,linewidth=1,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.text(x0, y1+50, f'License plate: {plate_num}', color = 'white')
ax.text(x0, y1+110, f'Probability: {plate_score*100:.2f} %', color = 'white')